# Evaluation

In [14]:
import json
import pprint
import os

In [10]:


unaffected = []
affected = []

patientID = '0'
data_type = 'Angles'
task = 'Sittingsponge2(Clone)'
healthy = 'HealthySegment'
stroke = 'StrokeSegment'
subtask = '1'

# Corrected paths
unaffected_path = './' + patientID + '/' + data_type + '/' + task  + healthy + '/' +  task  + healthy + subtask + '_0.json'
affected_path = './' + patientID + '/' + data_type + '/' + task  + stroke + '/' +  task  + stroke + subtask + '_0.json'

# Function to safely load JSON data
def load_json(file_path, data_list):
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r') as file:
                data_list.append(json.load(file))
        except json.JSONDecodeError:
            print(f"Error decoding JSON from {file_path}")
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
    else:
        print(f"File not found: {file_path}")

# Load unaffected and affected JSON data
load_json(unaffected_path, unaffected)
load_json(affected_path, affected)


In [15]:
def compare_joint_ranges(u_expt_jnt_dir, a_expt_jnt_dir):
    percentage_differences = {}
    for joint in u_expt_jnt_dir.keys():
        u_min, u_max = u_expt_jnt_dir[joint]
        a_min, a_max = a_expt_jnt_dir[joint]
        
        # Handle potential division by zero
        if u_min != 0:
            min_diff_pct = ((a_min - u_min) / abs(u_min)) * 100
        else:
            min_diff_pct = 0 if a_min == 0 else float('inf')
        
        if u_max != 0:
            max_diff_pct = ((a_max - u_max) / abs(u_max)) * 100 
        else:
            max_diff_pct = 0 if a_max == 0 else float('inf')
        
        percentage_differences[joint] = {
            'min_diff_pct': int(min_diff_pct),
            'max_diff_pct': int(max_diff_pct)
        }
    return percentage_differences

In [16]:
u_expt_jnt_dir = {}
a_expt_jnt_dir = {}


# initialize unaffected 
for dir in unaffected:
    J = list(dir.keys())    
    for j in J:
        prev_min, prev_max = u_expt_jnt_dir.get(j, (0, 0))
        current_min, current_max = min(dir[j]) / len(unaffected), max(dir[j]) / len(unaffected)
        u_expt_jnt_dir[j] = [prev_min + current_min, prev_max + current_max]
        


for dir in affected:
    J = list(dir.keys())
    for j in J:
        a_expt_jnt_dir[j] = [min(dir[j]), max(dir[j])]

percentage_differences = compare_joint_ranges(u_expt_jnt_dir, a_expt_jnt_dir)

pprint.pprint(percentage_differences)


{'leftElbowFlexionExtension': {'max_diff_pct': 18, 'min_diff_pct': -13},
 'leftIndexDIPFlexionExtension': {'max_diff_pct': 142, 'min_diff_pct': -30},
 'leftIndexMCPAbductionAdduction': {'max_diff_pct': 116, 'min_diff_pct': -11},
 'leftIndexMCPFlexionExtension': {'max_diff_pct': 336, 'min_diff_pct': -47},
 'leftIndexPIPFlexionExtension': {'max_diff_pct': 115, 'min_diff_pct': -68},
 'leftMiddleDIPFlexionExtension': {'max_diff_pct': 64, 'min_diff_pct': -37},
 'leftMiddleMCPAbductionAdduction': {'max_diff_pct': 447, 'min_diff_pct': -498},
 'leftMiddleMCPFlexionExtension': {'max_diff_pct': 261, 'min_diff_pct': -2421},
 'leftMiddlePIPFlexionExtension': {'max_diff_pct': 19, 'min_diff_pct': -41},
 'leftPinkyDIPFlexionExtension': {'max_diff_pct': 105, 'min_diff_pct': -89},
 'leftPinkyMCPAbductionAdduction': {'max_diff_pct': 14, 'min_diff_pct': -294},
 'leftPinkyMCPFlexionExtension': {'max_diff_pct': 51, 'min_diff_pct': -218},
 'leftPinkyPIPFlexionExtension': {'max_diff_pct': 103, 'min_diff_pct'

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the LSTM-based binary classification model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # LSTM output
        out, _ = self.lstm(x)  # We discard the hidden state and cell state
        # Take the output at the last timestep
        out = out[:, -1, :]
        # Fully connected layer to produce the final prediction
        out = self.fc(out)
        return out

# Initialize model, loss function, and optimizer
input_size = len(unaffected[0].keys())  # Number of joints
hidden_size = 64  # Number of LSTM units
num_layers = 2  # Number of LSTM layers
num_classes = 1  # Binary classification

model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary classification loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)


NameError: name 'X_train' is not defined